# **IEOR 4571 Fall 2020 Homework2 Report**


- Hu, Bo (uni: bh2569)
- Qin, Rui (uni: rq217)
- Yuan, Shuibenyang (uni: sy2938)

In [1]:
%load_ext autoreload
%autoreload 2 
import sys
sys.path.append('../')

# Objective

# The Sample

In [2]:
# todo how did we sample the data

In [3]:
from src.utils import loading, Spark
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pandas as pd
import numpy as np

## Load Sample

In [4]:
# create spark session
spark = Spark()

Spark UI address http://127.0.0.1:4041


In [5]:
# load sample data from '/data/raw/sample.csv'
datas = loading(spark, '../data/raw')
sample = datas['sample']

## Sample Objectives

In [6]:
print(f'''
            number of data points in the sample: {sample.count()},
            number of unique users in the sample: {sample.select('userId').distinct().count()},
            number of unique movies in the sample: {sample.select('movieId').distinct().count()},
            mean of number of movies a user rated:{sample.groupby('userId').agg(F.count('movieId').alias('cnt')).select(F.mean('cnt')).collect()[0][0]},
            mean of number of users a movie be rated: {sample.groupby('movieId').agg(F.count('userId').alias('cnt')).select(F.mean('cnt')).collect()[0][0]},
            average rating: {sample.select(F.mean('rating')).collect()[0][0]},
            standard deviation of rating: {sample.select(F.stddev('rating')).collect()[0][0]},
            average rating by user: {sample.groupby('userId').agg(F.mean('rating').alias('rating')).select(F.mean('rating')).collect()[0][0]},
            standard deviation of rating by user mean: {sample.groupby('userId').agg(F.mean('rating').alias('rating')).select(F.stddev('rating')).collect()[0][0]},
            average rating by movie: {sample.groupby('movieId').agg(F.mean('rating').alias('rating')).select(F.mean('rating')).collect()[0][0]},
            standard deviation of rating by movie mean: {sample.groupby('movieId').agg(F.mean('rating').alias('rating')).select(F.stddev('rating')).collect()[0][0]}
        ''')


            number of data points in the sample: 444289,
            number of unique users in the sample: 20000,
            number of unique movies in the sample: 1000,
            mean of number of movies a user rated:22.21445,
            mean of number of users a movie be rated: 444.289,
            average rating: 3.562069958968149,
            standard deviation of rating: 1.046784241962096,
            average rating by user: 3.685826601448731,
            standard deviation of rating by user mean: 0.5225862364055156,
            average rating by movie: 3.290922029378176,
            standard deviation of rating by movie mean: 0.5133362661920545
        


# Evaluation

## Cross Validation Setup

In [7]:
from src.evaluation import Evaluator, Cross_validate_als
from src.model_based import Als
import time

In [8]:
splits = loading(spark, '../data/interim')

In [9]:
print(list(splits.keys()))

['train_0.5_0.5', 'train_0.25_0.75', 'test_0.5_0.5', 'test_0.25_0.75', 'train_0.75_0.25', 'test_0.75_0.25']


In [10]:
# build evaluation pipeline
def evaluate(train, test, evaluators, model):
    print('training')
    start = time.time()
    model.fit(train)
    end = time.time()
    print(f'training time: {round(end - start, 2)} seconds')
    print('inferencing train set')
    start = time.time()
    train_pred = model.predict(train)
    end = time.time()
    print(f'inference time: {round(end - start, 2)} seconds')
    print('inferencing test set')
    start = time.time()
    test_pred = model.predict(test)
    end = time.time()
    print(f'inference time: {round(end - start, 2)} seconds')
    res = pd.DataFrame(np.zeros((len(evaluators),2)), columns = ['train', 'test'], index = evaluators.keys())
    for eva in evaluators.keys():
        res.loc[eva, 'train'] = evaluators[eva].evaluate(train_pred)
        res.loc[eva, 'test'] = evaluators[eva].evaluate(test_pred)
    return res

## Metrics

In [11]:
evaluators = {'rmse': Evaluator(metrics = 'rmse'), 
              'accuracy': Evaluator(metrics = 'accuracy'), 
              'coverage_2': Evaluator(metrics = 'converage_k', 
                                       ratingCol='rating', 
                                       predCol='prediction', 
                                       idCol='userId', 
                                       k=2)}

## Memory Based Collaborative Filtering

In [12]:
from src.memory_based import Memory_based_CF

### User Based Collaborative Filtering

In [22]:
cf = Memory_based_CF(spark, 'user')

In [23]:
%%time
result = []
for i in ['0.75_0.25', '0.5_0.5', '0.25_0.75']:
    train, test = splits['train_' + i], splits['test_' + i]
    res = evaluate(train, test, evaluators, cf)
    result.append(res)
ub_CF_res = pd.DataFrame(index=['rmse', 'accuracy', 'coverage_5'], columns = ['train', 'test', 'split'])
for i, j in zip(result, ['0.75_0.25', '0.5_0.5', '0.25_0.75']):
    i['split'] = j
    ub_CF_res = ub_CF_res.append(i)
ub_CF_res = ub_CF_res.dropna()

training
training time: 38.8 seconds
inferencing train set
inference time: 10.88 seconds
inferencing test set
inference time: 4.08 seconds
training
training time: 36.11 seconds
inferencing train set
inference time: 7.5 seconds
inferencing test set
inference time: 7.62 seconds
training
training time: 28.85 seconds
inferencing train set
inference time: 3.83 seconds
inferencing test set
inference time: 10.68 seconds
CPU times: user 3min 53s, sys: 32.7 s, total: 4min 25s
Wall time: 2min 32s


In [24]:
ub_CF_res

,train,test,split
rmse,0.822981,0.882472,0.75_0.25
accuracy,0.737826,0.705916,0.75_0.25
coverage_2,0.931907,0.707219,0.75_0.25
rmse,0.812562,0.903506,0.5_0.5
accuracy,0.744819,0.695154,0.5_0.5
coverage_2,0.880891,0.861922,0.5_0.5
rmse,0.789096,0.961355,0.25_0.75
accuracy,0.761302,0.674534,0.25_0.75
coverage_2,0.734384,0.920091,0.25_0.75


### Item Based Collaborative Filtering

In [ ]:
cf = Memory_based_CF(spark, 'item')

In [ ]:
%%time
result = []
for i in ['0.75_0.25', '0.5_0.5', '0.25_0.75']:
    train, test = splits['train_' + i], splits['test_' + i]
    res = evaluate(train, test, evaluators, cf)
    result.append(res)
ib_CF_res = pd.DataFrame(index=['rmse', 'accuracy', 'coverage_5'], columns = ['train', 'test', 'split'])
for i, j in zip(result, ['0.75_0.25', '0.5_0.5', '0.25_0.75']):
    i['split'] = j
    ib_CF_res = ib_CF_res.append(i)
ib_CF_res = ib_CF_res.dropna()

## Model Based Collaborative Filtering